In [1]:
import json
import jsonlines
from pyspark import SparkContext

FILTERING VIDEOS

In [2]:
keywords = {
    'climate change', 'global warming', 'greenhouse gases', 'rising sea levels',
    'extreme weather', 'temperature increase', 'climate action', 
    'carbon emissions', 'renewable energy', 'sustainable living',
    'climate science', 'climate crisis', 'climate adaptation',
    'natural disaster', 'extreme heat', 'melting ice cap', 'sea level rise',
    'biodiversity loss', 'deforestation', 'ocean acidification', 'climate policy', 
    'environmental policy',
    'disaster recovery', 'climate refugees',
    'carbon footprint', 'sustainable development', 'green technology',
    'renewable resource', 'eco-friendly',
    'ecosystem disruption', 'impact on climate',
    'paris agreement', 'climate awareness',
    'environmental justice', 'clean energy', 'zero carbon',
    'green infrastructure', 'ozone layer',
    'pollution', 'water scarcity', 'climate education'
}

keywords2 = {'olympics', 'olympic'}

##### Filter functions

In [3]:
#check if text or tags contain climate keywords:
#def idiot_filter(video):
#    return video['upload_date'] > '2015'


def about_climate(tags):
    tags_set = set(tags)
    return any(tag.strip().lower() in keywords2 for tag in tags_set)

def climate_text(text):
    lowercase_text = text.lower()
    return any(keyword in lowercase_text for keyword in keywords2)

def climate_related(video):
    return (climate_text(video['title']) or about_climate(video['tags']) or climate_text(video['description']))
    #return idiot_filter(video) and (climate_text(video['title']) or about_climate(video['tags']) or climate_text(video['description']))

##### We read and filter the video dataset

In [4]:
sc = SparkContext()

23/11/15 08:01:59 WARN Utils: Your hostname, sebastien-Inspiron-7391-2n1 resolves to a loopback address: 127.0.1.1; using 192.168.1.73 instead (on interface wlp0s20f3)
23/11/15 08:01:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/15 08:02:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
path_to_disk = '/Volumes/Maxtor'
path_to_disk_ubuntu = '/media/sebastien/Maxtor'
file_path = '/yt_metadata_en.jsonl'

raw_data = sc.textFile(path_to_disk_ubuntu + file_path)
video_dataset = raw_data.map(lambda x: json.loads(x))

In [6]:
#recent_videos = video_dataset.filter(idiot_filter)
#climate_videos = recent_videos.filter(climate_related)
climate_videos = video_dataset.filter(climate_related)
#climate_videos_collected = climate_videos.collect()

In [7]:
#climate_videos_path = "data/climate_videos_new.jsonl"

#with jsonlines.open(climate_videos_path, "w") as jsonl_file:
#    jsonl_file.write_all(climate_videos_collected)

##### Now, we count the number of video uploaded each day

This may be useful for our future analysis

In [8]:
nb_videos_by_date = video_dataset.map(lambda x: (x["upload_date"].split()[0], 1))

# Group by date and count occurrences
nb_videos_by_date = nb_videos_by_date.groupBy(lambda x: x[0]).map(lambda x: (x[0], len(x[1])))

# Collect the result
nb_videos_by_date_collected = nb_videos_by_date.collect()

ERROR:root:KeyboardInterrupt while sending command.            (710 + 8) / 2911]
Traceback (most recent call last):
  File "/home/sebastien/miniconda3/envs/2023/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/sebastien/miniconda3/envs/2023/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/sebastien/miniconda3/envs/2023/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
[Stage 0:=============>                                        (711 + 8) / 2911]
KeyboardInterrupt



In [ ]:
videos_by_date_path = "data/nb_videos_by_date.jsonl"
with jsonlines.open(videos_by_date_path, "w") as jsonl_file:
    jsonl_file.write_all(nb_videos_by_date_collected)
sc.stop()